In [1]:
%pip install numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.datasets import load_iris, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd

In [3]:
import numpy as np

np.random.seed(42)
n_amostras = 1000

# Colunas numéricas
idades = np.random.randint(18, 70, size=n_amostras)
salarios = np.random.randint(1500, 15000, size=n_amostras)

# Colunas categóricas
generos = np.random.choice(['M', 'F'], size=n_amostras)
estados_civis = np.random.choice(['solteiro', 'casado', 'divorciado'], size=n_amostras)

# Gerando risco baseado em regras lógicas (não aleatório)
riscos = []
for i in range(n_amostras):
    idade = idades[i]
    salario = salarios[i]
    estado = estados_civis[i]
    
    if salario < 4000 or idade < 23:
        risco = 'alto'
    elif salario < 8000 or estado == 'divorciado':
        risco = 'medio'
    else:
        risco = 'baixo'
    riscos.append(risco)

riscos = np.array(riscos)

# Combinar tudo
dados = np.column_stack((idades, estados_civis, salarios, generos, riscos))

# Separar features e target
X = dados[:, :-1]
y = dados[:, -1]

print("Exemplo de 5 amostras:")
print(dados[:5])
print("\nDistribuição de riscos:", np.unique(y, return_counts=True))


Exemplo de 5 amostras:
[['56' 'casado' '8382' 'M' 'baixo']
 ['69' 'solteiro' '3406' 'F' 'alto']
 ['46' 'divorciado' '4586' 'F' 'medio']
 ['32' 'divorciado' '9652' 'M' 'medio']
 ['60' 'divorciado' '7250' 'F' 'medio']]

Distribuição de riscos: (array(['alto', 'baixo', 'medio'], dtype='<U21'), array([271, 323, 406]))


In [4]:
X,y

(array([['56', 'casado', '8382', 'M'],
        ['69', 'solteiro', '3406', 'F'],
        ['46', 'divorciado', '4586', 'F'],
        ...,
        ['62', 'solteiro', '10484', 'M'],
        ['35', 'casado', '2291', 'F'],
        ['55', 'divorciado', '11501', 'M']], shape=(1000, 4), dtype='<U21'),
 array(['baixo', 'alto', 'medio', 'medio', 'medio', 'baixo', 'baixo',
        'medio', 'alto', 'alto', 'alto', 'medio', 'baixo', 'alto', 'medio',
        'medio', 'alto', 'baixo', 'alto', 'baixo', 'medio', 'baixo',
        'alto', 'alto', 'baixo', 'medio', 'medio', 'medio', 'baixo',
        'baixo', 'baixo', 'baixo', 'baixo', 'baixo', 'alto', 'baixo',
        'medio', 'medio', 'alto', 'medio', 'alto', 'medio', 'alto',
        'medio', 'medio', 'baixo', 'alto', 'baixo', 'alto', 'alto', 'alto',
        'medio', 'baixo', 'baixo', 'alto', 'baixo', 'medio', 'baixo',
        'medio', 'baixo', 'baixo', 'baixo', 'baixo', 'medio', 'alto',
        'medio', 'baixo', 'baixo', 'alto', 'alto', 'baixo', 'baixo',

In [5]:
X,y

for x,y_result in zip(X,y):
    print(x,'->',y_result)

['56' 'casado' '8382' 'M'] -> baixo
['69' 'solteiro' '3406' 'F'] -> alto
['46' 'divorciado' '4586' 'F'] -> medio
['32' 'divorciado' '9652' 'M'] -> medio
['60' 'divorciado' '7250' 'F'] -> medio
['25' 'casado' '14272' 'F'] -> baixo
['38' 'casado' '11767' 'F'] -> baixo
['56' 'divorciado' '10405' 'M'] -> medio
['36' 'solteiro' '2562' 'M'] -> alto
['40' 'solteiro' '3476' 'M'] -> alto
['28' 'casado' '2796' 'M'] -> alto
['28' 'solteiro' '7729' 'F'] -> medio
['41' 'casado' '9141' 'M'] -> baixo
['53' 'divorciado' '1625' 'M'] -> alto
['57' 'casado' '6132' 'F'] -> medio
['41' 'casado' '7788' 'F'] -> medio
['20' 'divorciado' '5096' 'F'] -> alto
['39' 'solteiro' '12975' 'M'] -> baixo
['19' 'casado' '7668' 'M'] -> alto
['41' 'casado' '11295' 'M'] -> baixo
['61' 'divorciado' '9061' 'F'] -> medio
['47' 'solteiro' '9758' 'F'] -> baixo
['55' 'casado' '1732' 'M'] -> alto
['19' 'divorciado' '11848' 'F'] -> alto
['38' 'casado' '10861' 'F'] -> baixo
['50' 'divorciado' '11850' 'F'] -> medio
['29' 'solteiro' 

In [6]:
class naiveBayes():
    
    def __init__(self):
        self.mean = []
        self.std = []
        self.probs = []
        self.probs_categoric = []
        self.classes = []
    
    # Função para detectar colunas numéricas e categóricas
    def separar_colunas(self,dados):
        colunas_numericas = []
        colunas_categoricas = []

        for i in range(dados.shape[1]):
            coluna = dados[:, i]
            # Tenta converter para float — se não der, é categórica
            try:
                coluna.astype(float)
                colunas_numericas.append(i)
            except ValueError:
                colunas_categoricas.append(i)
        
        return colunas_numericas, colunas_categoricas
    
    def fit_categoric(self, X, y):
        _, categoricas = self.separar_colunas(X)
        self.probs_categoric = []

        for x in categoricas:
            valores_possiveis = np.unique(X[:, x])
            probs_atributo = []

            for i, classe in enumerate(self.classes[0]):
                dados_classe = X[y == classe, x]
                vals, counts = np.unique(dados_classe, return_counts=True)
                
                prob_dict = dict(zip(vals, counts / self.classes[1][i]))

                probs = [prob_dict.get(v, 0) for v in valores_possiveis]
                probs_atributo.append(probs)

            self.probs_categoric.append((x, valores_possiveis, probs_atributo))
    
    def fit(self,X,y):
        self.classes = np.unique(y,return_counts=True)
        # print('Classes encontradas:',self.classes)
        for classe in self.classes[0]:
            numericas,_ = self.separar_colunas(X)
            
            counts_data = X[y==classe]
            counts_data_numerics = counts_data[:,numericas].astype(float)
            
            self.mean.append(np.mean(counts_data_numerics))
            self.std.append(np.std(counts_data_numerics))
            
            self.probs.append(len(counts_data_numerics) / len(X))
                
        self.fit_categoric(X, y)
        
    def predict(self, X):
        preds = []

        # identificar tipos de colunas
        numericas, categoricas = self.separar_colunas(X)

        for amostra in X:
            probs_classes = []

            # percorre cada classe (ex: baixo, medio, alto)
            for i, _ in enumerate(self.classes[0]):
                # --- Parte 1: probabilidade base da classe (P(classe))
                prob_total = self.probs[i]

                # --- Parte 2: features numéricas (Gaussiana)
                if numericas:
                    x_num = amostra[numericas].astype(float)
                    prob_gauss = np.prod(self.gauss(self.mean[i], self.std[i], x_num))
                    prob_total *= prob_gauss

                # --- Parte 3: features categóricas
                for col_idx, valores, probs_por_classe in self.probs_categoric:
                    valor = amostra[col_idx]
                    if valor in valores:
                        pos = np.where(valores == valor)[0][0]
                        prob_total *= probs_por_classe[i][pos]
                    else:
                        prob_total *= 1e-6  # suavização: valor não visto

                probs_classes.append(prob_total)

            # normaliza probabilidades (para interpretar como confiança)
            probs_classes = np.array(probs_classes)
            probs_classes_norm = probs_classes / probs_classes.sum()

            # classe com maior probabilidade
            pred_classe = self.classes[0][np.argmax(probs_classes)]
            preds.append((pred_classe, probs_classes_norm))

        return preds
    
    def gauss(self,mean,std,x):
        return (1 / (np.sqrt(2 * np.pi) * std)) * np.e ** (- (x - mean) ** 2 / (2 * std ** 2))

In [7]:
nB = naiveBayes()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

for x,y_result in zip(X_train,y_train):
    print(x,'->',y_result)

['36' 'casado' '6288' 'M'] -> medio
['40' 'casado' '10466' 'F'] -> baixo
['36' 'casado' '12972' 'F'] -> baixo
['59' 'solteiro' '4554' 'M'] -> medio
['54' 'solteiro' '3296' 'M'] -> alto
['22' 'casado' '9313' 'M'] -> alto
['50' 'solteiro' '14497' 'M'] -> baixo
['48' 'divorciado' '1653' 'M'] -> alto
['44' 'divorciado' '8937' 'F'] -> medio
['18' 'casado' '9215' 'M'] -> alto
['21' 'casado' '5703' 'M'] -> alto
['50' 'divorciado' '4528' 'F'] -> medio
['52' 'casado' '6636' 'F'] -> medio
['48' 'solteiro' '13816' 'M'] -> baixo
['68' 'solteiro' '5677' 'M'] -> medio
['69' 'casado' '5094' 'F'] -> medio
['19' 'divorciado' '7859' 'M'] -> alto
['47' 'solteiro' '5743' 'F'] -> medio
['30' 'casado' '5260' 'M'] -> medio
['49' 'solteiro' '12760' 'F'] -> baixo
['53' 'solteiro' '14734' 'M'] -> baixo
['56' 'casado' '6968' 'M'] -> medio
['58' 'solteiro' '13194' 'F'] -> baixo
['34' 'casado' '14900' 'M'] -> baixo
['45' 'casado' '5345' 'M'] -> medio
['29' 'solteiro' '11781' 'M'] -> baixo
['24' 'solteiro' '6257' '

In [9]:
nB.separar_colunas(X_train)

([0, 2], [1, 3])

In [10]:
nB.fit(X_train,y_train)

In [11]:
nB.probs_categoric

[(1,
  array(['casado', 'divorciado', 'solteiro'], dtype='<U21'),
  [[np.float64(0.3333333333333333),
    np.float64(0.32275132275132273),
    np.float64(0.3439153439153439)],
   [np.float64(0.4838709677419355), 0, np.float64(0.5161290322580645)],
   [np.float64(0.2108843537414966),
    np.float64(0.5578231292517006),
    np.float64(0.23129251700680273)]]),
 (3,
  array(['F', 'M'], dtype='<U21'),
  [[np.float64(0.5079365079365079), np.float64(0.49206349206349204)],
   [np.float64(0.5483870967741935), np.float64(0.45161290322580644)],
   [np.float64(0.5), np.float64(0.5)]])]

In [12]:
for atributo in nB.probs_categoric:
    print(atributo)
    # for item in atributo:
    #     print(item)

(1, array(['casado', 'divorciado', 'solteiro'], dtype='<U21'), [[np.float64(0.3333333333333333), np.float64(0.32275132275132273), np.float64(0.3439153439153439)], [np.float64(0.4838709677419355), 0, np.float64(0.5161290322580645)], [np.float64(0.2108843537414966), np.float64(0.5578231292517006), np.float64(0.23129251700680273)]])
(3, array(['F', 'M'], dtype='<U21'), [[np.float64(0.5079365079365079), np.float64(0.49206349206349204)], [np.float64(0.5483870967741935), np.float64(0.45161290322580644)], [np.float64(0.5), np.float64(0.5)]])


In [13]:
nB.mean, nB.std, nB.probs,nB.probs_categoric,nB.classes

([np.float64(2363.9603174603176),
  np.float64(5729.894009216589),
  np.float64(4119.503401360545)],
 [np.float64(3321.556264305004),
  np.float64(5853.936272917862),
  np.float64(4672.47186671322)],
 [0.27, 0.31, 0.42],
 [(1,
   array(['casado', 'divorciado', 'solteiro'], dtype='<U21'),
   [[np.float64(0.3333333333333333),
     np.float64(0.32275132275132273),
     np.float64(0.3439153439153439)],
    [np.float64(0.4838709677419355), 0, np.float64(0.5161290322580645)],
    [np.float64(0.2108843537414966),
     np.float64(0.5578231292517006),
     np.float64(0.23129251700680273)]]),
  (3,
   array(['F', 'M'], dtype='<U21'),
   [[np.float64(0.5079365079365079), np.float64(0.49206349206349204)],
    [np.float64(0.5483870967741935), np.float64(0.45161290322580644)],
    [np.float64(0.5), np.float64(0.5)]])],
 (array(['alto', 'baixo', 'medio'], dtype='<U21'), array([189, 217, 294])))

In [14]:
y_pred = nB.predict(X_test)

In [15]:
y_pred

[(np.str_('medio'), array([0.01245553, 0.        , 0.98754447])),
 (np.str_('medio'), array([0.14628197, 0.        , 0.85371803])),
 (np.str_('baixo'), array([0.13309665, 0.55106887, 0.31583448])),
 (np.str_('medio'), array([0.30529053, 0.        , 0.69470947])),
 (np.str_('medio'), array([0.03797121, 0.        , 0.96202879])),
 (np.str_('alto'), array([0.49777058, 0.23399439, 0.26823504])),
 (np.str_('baixo'), array([0.32301364, 0.37197941, 0.30500695])),
 (np.str_('medio'), array([0.01360161, 0.        , 0.98639839])),
 (np.str_('alto'), array([0.51995268, 0.24225107, 0.23779625])),
 (np.str_('baixo'), array([0.10744238, 0.57318706, 0.31937056])),
 (np.str_('baixo'), array([0.16896158, 0.50503998, 0.32599845])),
 (np.str_('alto'), array([0.57184533, 0.2112197 , 0.21693497])),
 (np.str_('medio'), array([0.02834251, 0.        , 0.97165749])),
 (np.str_('alto'), array([0.47655001, 0.26994871, 0.25350128])),
 (np.str_('alto'), array([0.36985737, 0.3436031 , 0.28653953])),
 (np.str_('baix

In [16]:
y_pred = [pred[0] for pred in y_pred]

In [17]:
accuracy_score(y_test, y_pred)

0.71

In [18]:
confusion_matrix(y_test, y_pred)

array([[ 38,  14,  30],
       [  0, 106,   0],
       [ 29,  14,  69]])